<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/Run_Eval_on_Saved_Mixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [2]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 47.2 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filenam

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [4]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [5]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [6]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [7]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [8]:
from omegaconf import OmegaConf
config = OmegaConf.load('config.yaml')
print("default source sep model: ", config.separator.model)

default source sep model:  demucs


Set input directory

In [9]:
reference_folder = Path("Val_Ref_Tracks")
enhanced_mix_folder = Path("Val_enhanced_signals_openunmix_100")

# load the audio files in provided path

In [10]:
enhanced_files = [] # initialize list of enhanced audio files
reference_files = [] # initialize list of reference files
for file in os.listdir(enhanced_mix_folder): # iterate over all files in the directory
    if file.endswith('.flac'): # if the file is an audio file
        enhanced_files.append(os.path.join(enhanced_mix_folder, file)) # add the file to the list of audio files
        reference_files.append(os.path.join(reference_folder, file.replace('.flac','_ref.flac'))) # add the corresponding ref file to the list of ref files

In [11]:
print(enhanced_files[0])
print(reference_files[0])
print(len(enhanced_files))

Val_enhanced_signals_openunmix_100/scene_50001_L5048_remix.flac
Val_Ref_Tracks/scene_50001_L5048_remix_ref.flac
100


Load listener data

In [12]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation'


In [13]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [14]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [15]:
print(scene_listener_pairs[0])

('scene_50001', 'L5020')


# Load functions required for audio evaluation

Add functions and variables needed for evaluation and scoring

In [16]:
scores_headers = [
    "scene",
    "song",
    "listener",
    "left_score",
    "right_score",
    "score",
]


results_file = ResultsFile(
            "Val_scores_openunmix.csv",
            header_columns=scores_headers,
)

# Process Audio - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset) - HAAQI is very very slow

In [17]:
from recipes.cad_icassp_2024.baseline.evaluate import level_normalisation
previous_song = ""
num_scenes = len(scene_listener_pairs)
nrun = 100 # change to num_scenes for full dataset


for idx in range(num_scenes):
    # Extract track, listener, scene info from file name
    file_name = enhanced_files[idx]
    sname = file_name.split("/", 1)[1].split("_L",1)[0]
    lname = "L" +  file_name.split("/", 1)[1].split("_L",1)[1].split("_")[0]

    # Get the listener's audiogram
    listener = listener_dict[lname]
    scene = scenes[sname]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    print(sname)
    print(lname)

    # load audio files for HAAQI
    reference_mixture = read_signal(
            filename=reference_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = read_signal(
            filename=enhanced_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    # Evaluate - compare with the generated mixes
    # Compute the scores
    left_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 0],
            config.remix_sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 0], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_left,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 0])),
    )

    right_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 1],
            config.remix_sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 1], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_right,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 1])),
    )

        # Save scores
    results_file.add_result(
        {
            "scene": sname,
            "song": song_name,
            "listener": listener.id,
            "left_score": left_score,
            "right_score": right_score,
            "score": float(np.mean([left_score, right_score])),
        }
    )

    print(right_score, left_score)


scene_50001
L5048
0.12287436914434821 0.09761942408743415
scene_50002
L5011
0.14389393992138047 0.1370799638431106
scene_50002
L5024
0.10600208668436481 0.12723895439132377
scene_50003
L5027
0.14575217240905747 0.11878847412156862
scene_50003
L5053
0.15246653155926657 0.14887067157029188
scene_50004
L5009
0.08095481004163457 0.10708703860946145
scene_50004
L5013
0.09725590232575983 0.08850369655856152
scene_50005
L5091
0.15574438359892961 0.14756711167004002
scene_50005
L5005
0.11726910484962005 0.10392585967208245
scene_50006
L5047
0.15339819227057738 0.11892006401175813
scene_50006
L5031
0.11313654883679627 0.11449159595559198
scene_50007
L5024
0.10315829262413548 0.12383451192377654
scene_50007
L5018
0.10181686946543919 0.10221146413806916
scene_50008
L5007
0.09218558296031967 0.11131713010894141
scene_50008
L5086
0.15553313172217467 0.14948178836715667
scene_50009
L5066
0.1539001131706696 0.14129842395514863
scene_50009
L5018
0.12461441917846332 0.12813123836908472
scene_50010
L506

0.09339988054291372 0.1364560934258882
scene_50020
L5014
0.10491531949665256 0.09864347951793824
scene_50020
L5039
0.1500863785442924 0.11965088915863334
scene_50021
L5063
0.1471044412985593 0.11754418311308781
scene_50021
L5064
0.1306025163875729 0.12412285163471937
scene_50022
L5079
0.13555858284748443 0.15163369288556064
scene_50022
L5078
0.14648232409574888 0.13665084040731648
scene_50023
L5042
0.11019747729841545 0.110748636612731
scene_50023
L5005
0.09842632777138502 0.10114873544731796
scene_50024
L5027
0.12101856007614673 0.0935127265955871
scene_50024
L5085
0.10732241522172767 0.09131945352880294
scene_50025
L5063
0.14874958205667255 0.13478923224509898
scene_50025
L5079
0.14376145680697022 0.14375278314111004
scene_50026
L5069
0.1116725288274936 0.10563338124388433
scene_50026
L5041
0.14769612704340288 0.10761119656523044
scene_50027
L5069
0.13023016876104754 0.13263852815730498
scene_50027
L5009
0.12446983515989454 0.1536368883530894
scene_50028
L5076
0.1129691266961212 0.12

0.13496636550684044 0.12014870516661323
scene_50042
L5023
0.12708714405004948 0.14279018903929686
scene_50043
L5079
0.1513087544723264 0.16192498215898843
scene_50043
L5062
0.16774553908295853 0.15147267100926062
scene_50044
L5086
0.14276162220280347 0.1497416565148355
scene_50044
L5071
0.16350590118195724 0.1402731445689984
scene_50045
L5039
0.1636676719713931 0.1501463910962775
scene_50045
L5045
0.17212117218426443 0.15843326949534914
scene_50046
L5999
0.1647676770765567 0.15411410802453596
scene_50046
L5019
0.13368601489058535 0.10763371344900571
scene_50047
L5043
0.12688195924063747 0.12022929530940413
scene_50047
L5078
0.16373772638424666 0.1440794453560082
scene_50048
L5014
0.10730643072163248 0.11442957217032876
scene_50048
L5005
0.11692351913110524 0.10976035944705338
scene_50049
L5057
0.1321521905627405 0.1253089072505499
scene_50049
L5020
0.1536271079959197 0.13601115414049786
scene_50050
L5029
0.13783110160832407 0.1270595071622017
scene_50050
L5076
0.16656038695163786 0.160

LibsndfileError: ignored

# Implementing classic SDR eval

In [ ]:
!pip install mir_eval
from mir_eval import separation
separation.bss_eval_sources(song1, song2)[0].mean()